In [1]:
import pandas as pd
from tqdm import tqdm
import os

In [2]:
data_train = pd.read_csv("df_train.csv")
data_test = pd.read_csv("df_test.csv")
data_val = pd.read_csv("df_val.csv")
X_train, Y_train = data_train.drop('teamVictory',axis=1),data_train[['teamVictory']]


In [3]:
X_train

,Unnamed: 0.1,Unnamed: 0,gameId,matchId,gameVersion,gameDuration,team_100_gold,team_200_gold,Player_id,Player_1_ban,...,Player_10_role,Player_10_team,Player_10_victory,Player_10_position,Player_10_time_game,Player_10_gold,Player_10_xp,Player_10_dmg_dealt,Player_10_dmg_taken,Player_10_time_ccing
0,0,0,398776824,RU_398776824,12.12.450.4196,1837,65405,58915,10,-1,...,SUPPORT,200,0,UTILITY,1837,7847,10195,6120,24057,28
1,1,0,4363972028,NA1_4363972028,12.12.450.4196,1566,48274,55955,10,876,...,SUPPORT,200,1,UTILITY,1566,7619,9623,4525,8963,41
2,2,0,3168495552,EUN1_3168495552,12.12.450.4196,1572,48206,61837,10,122,...,NONE,200,1,UTILITY,1572,13997,12369,28012,13220,27
3,3,0,1314237530,TR1_1314237530,12.12.450.4196,1901,72646,64186,10,200,...,SUPPORT,200,0,UTILITY,1901,8702,12848,15059,33510,63
4,4,0,1193729142,LA2_1193729142,12.12.450.4196,1485,36338,55458,10,111,...,SOLO,200,1,UTILITY,1485,8617,11778,8615,6494,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86631,86631,0,5964301583,EUW1_5964301583,12.12.450.4196,926,23857,35131,10,39,...,SUPPORT,200,1,UTILITY,926,4869,4186,3006,4811,10
86632,86632,0,5953381872,EUW1_5953381872,12.12.450.4196,1235,32813,38548,10,85,...,SUPPORT,200,1,UTILITY,1235,5714,7042,3500,5675,38
86633,86633,0,2553762406,BR1_2553762406,12.12.450.4196,1619,55793,48927,10,7,...,SUPPORT,200,0,UTILITY,1619,6664,8706,7787,12810,25
86634,86634,0,5950633298,EUW1_5950633298,12.12.450.4196,1117,40186,26627,10,62,...,SUPPORT,200,0,UTILITY,1117,3897,3676,2682,7051,11


In [4]:
df = X_train.loc[(X_train['Player_1_position']=='TOP') 
& (X_train['Player_2_position']=='JUNGLE')
&(X_train['Player_3_position']=='MIDDLE') 
& (X_train['Player_4_position']=='BOTTOM')
&(X_train['Player_5_position']=='UTILITY') 
& (X_train['Player_6_position']=='TOP') 
& (X_train['Player_7_position']=='JUNGLE')
&(X_train['Player_8_position']=='MIDDLE') 
& (X_train['Player_9_position']=='BOTTOM')
&(X_train['Player_10_position']=='UTILITY')]

In [5]:
def ChampCounterRelations(df):
    top_counter = df[['Player_1_pick','Player_1_gold','Player_6_pick','Player_6_gold']].groupby(['Player_1_pick','Player_6_pick']).mean().reset_index()
    top_counter['counter_Score'] = top_counter.apply(lambda x: x['Player_1_gold']-x['Player_6_gold'], axis=1)
    jug_counter = df[['Player_2_pick','Player_2_gold','Player_7_pick','Player_7_gold']].groupby(['Player_2_pick','Player_7_pick']).mean().reset_index()
    jug_counter['counter_Score'] = jug_counter.apply(lambda x: x['Player_2_gold']-x['Player_7_gold'], axis=1)
    mid_counter = df[['Player_3_pick','Player_3_gold','Player_8_pick','Player_8_gold']].groupby(['Player_3_pick','Player_8_pick']).mean().reset_index()
    mid_counter['counter_Score'] = mid_counter.apply(lambda x: x['Player_3_gold']-x['Player_8_gold'], axis=1)
    bot_counter = df[['Player_4_pick','Player_4_gold','Player_9_pick','Player_9_gold']].groupby(['Player_4_pick','Player_9_pick']).mean().reset_index()
    bot_counter['counter_Score'] = bot_counter.apply(lambda x: x['Player_4_gold']-x['Player_9_gold'], axis=1)
    sup_counter = df[['Player_5_pick','Player_5_gold','Player_10_pick','Player_10_gold']].groupby(['Player_5_pick','Player_10_pick']).mean().reset_index()
    sup_counter['counter_Score'] = sup_counter.apply(lambda x: x['Player_5_gold']-x['Player_10_gold'], axis=1)
    return top_counter,jug_counter,mid_counter,bot_counter,sup_counter


In [6]:
ChampCounterRelations(df)[0]

,Player_1_pick,Player_6_pick,Player_1_gold,Player_6_gold,counter_Score
0,1,2,12489.00,15984.000000,-3495.000000
1,1,14,12031.00,9220.000000,2811.000000
2,1,23,9028.00,9979.333333,-951.333333
3,1,24,10038.00,11873.250000,-1835.250000
4,1,27,12942.00,13222.000000,-280.000000
...,...,...,...,...,...
6268,887,876,9077.25,9913.250000,-836.000000
6269,887,888,9842.00,6524.000000,3318.000000
6270,888,14,13454.00,16662.000000,-3208.000000
6271,888,41,7580.00,10675.000000,-3095.000000
